In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt


In [2]:
IMAGE_SIZE = 30
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'Train',
    shuffle = True,
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    seed = 123,
)

Found 39209 files belonging to 43 classes.


In [4]:
cls_names = dataset.class_names
cls_names

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '5',
 '6',
 '7',
 '8',
 '9']

In [5]:
class_names_in_num = list(cls_names)
class_names_in_num = [int(i) for i in class_names_in_num]
class_names_in_num

[0,
 1,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 2,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 3,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 4,
 40,
 41,
 42,
 5,
 6,
 7,
 8,
 9]

In [6]:
# Classes of trafic signs
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }


In [7]:
CLASS_NAMES = []
for i in class_names_in_num:
    CLASS_NAMES.append(classes[i])
CLASS_NAMES

['Speed limit (20km/h)',
 'Speed limit (30km/h)',
 'No passing veh over 3.5 tons',
 'Right-of-way at intersection',
 'Priority road',
 'Yield',
 'Stop',
 'No vehicles',
 'Veh > 3.5 tons prohibited',
 'No entry',
 'General caution',
 'Dangerous curve left',
 'Speed limit (50km/h)',
 'Dangerous curve right',
 'Double curve',
 'Bumpy road',
 'Slippery road',
 'Road narrows on the right',
 'Road work',
 'Traffic signals',
 'Pedestrians',
 'Children crossing',
 'Bicycles crossing',
 'Speed limit (60km/h)',
 'Beware of ice/snow',
 'Wild animals crossing',
 'End speed + passing limits',
 'Turn right ahead',
 'Turn left ahead',
 'Ahead only',
 'Go straight or right',
 'Go straight or left',
 'Keep right',
 'Keep left',
 'Speed limit (70km/h)',
 'Roundabout mandatory',
 'End of no passing',
 'End no passing veh > 3.5 tons',
 'Speed limit (80km/h)',
 'End of speed limit (80km/h)',
 'Speed limit (100km/h)',
 'Speed limit (120km/h)',
 'No passing']

In [8]:
len(dataset)

1226

In [9]:
for images, labels in dataset.take(1):
    print(images.shape)
    print(labels.numpy())


(32, 30, 30, 3)
[ 5 17  9  5  3 18 15 23 32 39 42  5 36 28 23  1 18 12 40 38  4 32 23 42
 40  6  1 38 41 21 40  2]


In [10]:
def get_train_test_split(ds, train_split = 0.8, val_split = 0.1, test_split = 0.1, shuffle = True, shuffle_size = 1000):
    assert(train_split + val_split + test_split) == 1
    if shuffle:
        ds.shuffle(shuffle_size, seed = 12)
    ds_size = len(ds)
    train_size = int(ds_size*train_split)
    val_size = int(ds_size*val_split)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_train_test_split(ds = dataset)

print(f"the number of batches for training: {len(train_ds)}")
print(f"the number of batches for validation: {len(val_ds)}")
print(f"the number of batches for testing: {len(test_ds)}")

the number of batches for training: 980
the number of batches for validation: 122
the number of batches for testing: 124


In [11]:
train_ds = train_ds.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [13]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1/255),
])

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

INPUT_SHAPE = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = len(CLASS_NAMES)  # 43

model = tf.keras.models.Sequential([
    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(32, kernel_size = (3,3), activation = "relu", input_shape = INPUT_SHAPE), # 30 -(3-1) = 28
    layers.MaxPooling2D(pool_size = (2,2)), # 28/2 = 14

    layers.Conv2D(64, kernel_size = (3,3), activation= 'relu'), # 14 -(3-1) = 12
    layers.MaxPooling2D(pool_size=(2,2)), # 12/2 = 6

    layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'), # 6-(3-1) = 4
    layers.MaxPooling2D(pool_size=(2,2)), # 4/2 = 2


    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(n_classes, activation='softmax')
])

model.build(input_shape = INPUT_SHAPE)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (32, 30, 30, 3)           0         
_________________________________________________________________
sequential_4 (Sequential)    (32, 30, 30, 3)           0         
_________________________________________________________________
conv2d_6 (Conv2D)            (32, 28, 28, 32)          896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (32, 14, 14, 32)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (32, 12, 12, 64)          18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (32, 6, 6, 64)            0         
_________________________________________________________________
conv2d_8 (Conv2D)            (32, 4, 4, 64)           

In [ ]:
model.compile(optimizer = "adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False), metrics = ["accuracy"])
history = model.fit(
    train_ds,
    validation_data = val_ds,
    batch_size = BATCH_SIZE,
    verbose = 1,
    epochs = 50
)

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers, models

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 256


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 512
EPOCHS = 50
CHANNELS = 3

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Train",
    batch_size = BATCH_SIZE,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    
)